In [9]:
class Interpreter:
    def __init__(self):
        self.stack = [] # [7, 4, 8]
        self.environment = {} # {"x": 7, "y": 4, "z": 8}

    def LOAD_VALUE(self, number):
        self.stack.append(number)
    
    def STORE_NAME(self, name):
        value = self.stack.pop()
        self.environment[name] = value

    def LOAD_NAME(self, name):
        value = self.environment[name]
        self.stack.append(value)
    
    def PRINT_ANSWER(self):
        answer = self.stack.pop()
        print(answer)
    
    def ADD_TWO_VALUES(self):
        first_num = self.stack.pop()
        second_num = self.stack.pop()
        total = first_num + second_num
        self.stack.append(total)
    
    def parse_argument(self, instruction, argument, what_to_execute):
        numbers = ["LOAD_VALUE"]
        names = ["LOAD_NAME", "STORE_NAME"]
        if instruction in numbers:
            argument = what_to_execute["numbers"][argument]
        elif instruction in names:
            argument = what_to_execute["names"][argument]
        return argument
    
    def run_code(self, what_to_execute):
        instructions = what_to_execute["instructions"]
        for each_step in instructions:
            instruction, argument = each_step
            argument = self.parse_argument(instruction, argument, what_to_execute)

            if instruction == "LOAD_VALUE":
                self.LOAD_VALUE(argument)
            elif instruction == "ADD_TWO_VALUES":
                self.ADD_TWO_VALUES()
            elif instruction == "PRINT_ANSWER":
                self.PRINT_ANSWER()
            elif instruction == "STORE_NAME":
                self.STORE_NAME(argument)
            elif instruction == "LOAD_NAME":
                self.LOAD_NAME(argument)
    
    def execute(self, what_to_execute):
        instructions = what_to_execute["instructions"]
        for each_step in instructions:
            instruction, argument = each_step
            argument = self.parse_argument(instruction, argument, what_to_execute)
            bytecode_method = getattr(self, instruction)
            
            if argument is None:
                bytecode_method()
            else:
                bytecode_method(argument)

7 + 5

instructions are:

In [3]:
what_to_execute = {
    "instructions": [("LOAD_VALUE", 0),  # the first number
                     ("LOAD_VALUE", 1),  # the second number
                     ("ADD_TWO_VALUES", None),
                     ("PRINT_ANSWER", None)],
    "numbers": [7, 5] }

In [14]:
interpreter = Interpreter()
interpreter.run_code(what_to_execute)

3


In [15]:
what_to_execute = {
    "instructions": [("LOAD_VALUE", 0),
                        ("LOAD_VALUE", 1),
                        ("ADD_TWO_VALUES", None),
                        ("LOAD_VALUE", 2),
                        ("ADD_TWO_VALUES", None),
                        ("PRINT_ANSWER", None)],
    "numbers": [7, 5, 8] }

In [16]:
interpreter.run_code(what_to_execute)

20


In [17]:
what_to_execute = {
    "instructions": [("LOAD_VALUE", 0),
                        ("STORE_NAME", 0),
                        ("LOAD_VALUE", 1),
                        ("STORE_NAME", 1),
                        ("LOAD_NAME", 0),
                        ("LOAD_NAME", 1),
                        ("ADD_TWO_VALUES", None),
                        ("PRINT_ANSWER", None)],
    "numbers": [1, 2],
    "names":   ["a", "b"] }

In [19]:
interpreter.execute(what_to_execute)

3


Real Python Bytecode

In [20]:
def cond():
    x = 5
    if x > 5:
        print("True")
    else:
        print("False")

In [21]:
cond.__code__.co_code

b'd\x01}\x00|\x00d\x01k\x04r\x16t\x00d\x02\x83\x01\x01\x00n\x08t\x00d\x03\x83\x01\x01\x00d\x00S\x00'

In [22]:
list(cond.__code__.co_code)

[100,
 1,
 125,
 0,
 124,
 0,
 100,
 1,
 107,
 4,
 114,
 22,
 116,
 0,
 100,
 2,
 131,
 1,
 1,
 0,
 110,
 8,
 116,
 0,
 100,
 3,
 131,
 1,
 1,
 0,
 100,
 0,
 83,
 0]

In [23]:
import dis 

dis.dis(cond)

  2           0 LOAD_CONST               1 (5)
              2 STORE_FAST               0 (x)

  3           4 LOAD_FAST                0 (x)
              6 LOAD_CONST               1 (5)
              8 COMPARE_OP               4 (>)
             10 POP_JUMP_IF_FALSE       22

  4          12 LOAD_GLOBAL              0 (print)
             14 LOAD_CONST               2 ('True')
             16 CALL_FUNCTION            1
             18 POP_TOP
             20 JUMP_FORWARD             8 (to 30)

  6     >>   22 LOAD_GLOBAL              0 (print)
             24 LOAD_CONST               3 ('False')
             26 CALL_FUNCTION            1
             28 POP_TOP
        >>   30 LOAD_CONST               0 (None)
             32 RETURN_VALUE


In [29]:
dis.opname[131]

'CALL_FUNCTION'

In [30]:
def loop():
    x = 1
    while x < 5:
        x = x + 1

In [31]:
dis.dis(loop)

  2           0 LOAD_CONST               1 (1)
              2 STORE_FAST               0 (x)

  3           4 SETUP_LOOP              20 (to 26)
        >>    6 LOAD_FAST                0 (x)
              8 LOAD_CONST               2 (5)
             10 COMPARE_OP               0 (<)
             12 POP_JUMP_IF_FALSE       24

  4          14 LOAD_FAST                0 (x)
             16 LOAD_CONST               1 (1)
             18 BINARY_ADD
             20 STORE_FAST               0 (x)
             22 JUMP_ABSOLUTE            6
        >>   24 POP_BLOCK
        >>   26 LOAD_CONST               0 (None)
             28 RETURN_VALUE


In [32]:
def func5():
    return 3 + 4

def func6():
    result = 3 + 4
    return result


In [33]:
dis.dis(func5)

  2           0 LOAD_CONST               1 (7)
              2 RETURN_VALUE


In [34]:
dis.dis(func6)

  5           0 LOAD_CONST               1 (7)
              2 STORE_FAST               0 (result)

  6           4 LOAD_FAST                0 (result)
              6 RETURN_VALUE
